In [1]:
import sys
import arcpy
from arcpy.sa import *
from arcpy import env

In [2]:
# nadpisywanie plików
arcpy.env.overwriteOutput=True

In [3]:
# sprawdzanie rozszerzeń
# https://pro.arcgis.com/en/pro-app/latest/arcpy/functions/checkextension.htm
arcpy.CheckOutExtension("Spatial")
arcpy.CheckOutExtension("3D")

'CheckedOut'

In [4]:
# ustawienia środowiska pracy
# data paths/ścieżka katalogu
arcpy.env.workspace = r"A:\Studia\analizy_przestrzenne\MiloszWojciechowski\projekt1\Wyniki"

# processing extent (obszar na którym będą wykonywane analizy)
arcpy.env.extent = r"A:\Studia\analizy_przestrzenne\MiloszWojciechowski\projekt1\swieradow_buffer.shp"

# output coordinate system
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("ETRS_1989_Poland_CS92")

# cell size
arcpy.env.cellSize = 1

In [5]:
#przypisanie warstw do zmiennych
budynki = "bubd"
rzeki = "swrs"
drogi = "skdr"
lasy = "ptlz"
wody_powierzchniowe = "ptwp"
gazociag = "gazociag"
gleby = "gleby_gm_swieradow"
hotele = "hotele"
gmina = "swieradow"

<Result 'A:\\Studia\\analizy_przestrzenne\\MiloszWojciechowski\\projekt1\\Wyniki\\swieradowzdroj_Buffer.shp'>

# Opracowywanie kryteriów

1.Odległośc od konkurencji: min. 400m

In [ ]:
#Stworzenie mapy odegłości, a następnie przydatności terenu
euclidean_distance = EucDistance(hotele)
mapa_odleglosci = ExtractByMask(euclidean_distance,gmina)
hotele_kryterium1 = FuzzyMembership(mapa_odleglosci,FuzzyLinear(400,500))

2.Odległość od budynków mieszkalnych: 25-150 m

In [6]:
# Przycięcie warstwy budynków
budynkiMieszkalne = arcpy.management.SelectLayerByLocation(budynki, "INTERSECT", "swieradow_buffer", None, "NEW_SELECTION", "NOT_INVERT")
budynkiMieszkalne = arcpy.management.SelectLayerByAttribute(budynki, "SUBSET_SELECTION", "X_KOD = 'BUBD01' Or X_KOD = 'BUBD03' Or X_KOD = 'BUBD04'", None)
arcpy.management.CopyFeatures(budynkiMieszkalne, r"budynki_mieszkalne")

#stworzenie mapy odległości i przydatności terenu
euclidean_distance = EucDistance("budynki_mieszkalne")
mapa_odleglosci = ExtractByMask(euclidean_distance,gmina)
mapa_bez_mieszkan1 = FuzzyMembership(mapa_odleglosci,FuzzyLinear(0,25))
mapa_bez_mieszkan2 = FuzzyMembership(mapa_odleglosci,FuzzyLinear(250,190))
budynkiMieszkalne_kryterium2 = mapa_bez_mieszkan1 * mapa_bez_mieszkan2

# Wyczyszczenie selecta
arcpy.SelectLayerByAttribute_management(budynkiMieszkalne, "CLEAR_SELECTION")

<Result 'bubd'>

3.Odległość od istniejących dróg: 15-100 m

In [7]:
# Przycięcie warstwy dróg
drogi = arcpy.analysis.Clip(drogi, "swieradow_buffer", r"drogi", None)

#Opracowanie kryterium
euclidean_distance = EucDistance(drogi)
mapa_odleglosci = ExtractByMask(euclidean_distance,gmina)
mapa_drogi1 = FuzzyMembership(mapa_odleglosci,FuzzyLinear(0,15))
mapa_drogi2 = FuzzyMembership(mapa_odleglosci,FuzzyLinear(210,150))
drogi_kryterium3 = mapa_drogi1 * mapa_drogi2

4.Odległość od rzek i zbiorników wodnych: min. 20m

In [8]:
#Opracowanie 1 części kryterium
euclidean_distance = EucDistance("rzeki")
mapa_odleglosci = ExtractByMask(euclidean_distance,gmina)
rzeki_ostre = FuzzyMembership(mapa_odleglosci,FuzzyLinear(20,20.1))
rzeki_miekkie = FuzzyMembership(mapa_odleglosci,FuzzyLinear(900,500))
rzeki_kryterium4 = rzeki_ostre*rzeki_miekkie

#Przycięcie warstwy wód powierzchniowych
wody_powierzchniowe = arcpy.management.SelectLayerByLocation(wody_powierzchniowe, "INTERSECT", "swieradow_buffer", None, "NEW_SELECTION", "NOT_INVERT")
arcpy.management.CopyFeatures(wody_powierzchniowe, r"wody_powierzchniowe")

#Opracowanie 2 części kryterium
euclidean_distance = EucDistance("wody_powierzchniowe")
mapa_odleglosci = ExtractByMask(euclidean_distance,gmina)
wody_ostre = FuzzyMembership(mapa_odleglosci,FuzzyLinear(20,20.1))
wody_miekkie = FuzzyMembership(mapa_odleglosci,FuzzyLinear(900,500))
wody_kryterium4 = wody_ostre*wody_miekkie

# Wyczyszczenie selecta
arcpy.SelectLayerByAttribute_management(wody_powierzchniowe, "CLEAR_SELECTION")

<Result 'ptwp'>

5.Pokrycie terenu: nie w lesie

In [ ]:
# Przycięcie warstwy lasów
lasy = arcpy.management.SelectLayerByLocation(lasy, "INTERSECT", "swieradow_buffer", None, "NEW_SELECTION", "NOT_INVERT")
lasy = arcpy.management.SelectLayerByAttribute(lasy, "SUBSET_SELECTION", "RODZAJ = 'las' Or RODZAJ = 'zagajnik'", None)
arcpy.management.CopyFeatures(lasy, r"lasy")

#Opracowanie kryterium
arcpy.conversion.FeatureToRaster(lasy, "RODZAJ", r"mapa_lasow", 1)
arcpy.ddd.Reclassify("mapa_lasow", "RODZAJ", "las 0;zagajnik 0;NODATA 1", r"lasy_raster", "DATA")
lasy_kryterium5 = ExtractByMask("lasy_raster",gmina)

# Wyczyszczenie selecta
arcpy.SelectLayerByAttribute_management(lasy, "CLEAR_SELECTION")

6.Nachylenie stoków: max. 20%

In [ ]:
#Utworzenie mapy nachylenia stoków
nachylenie = Slope("gmina_swieradow_raster.tif", "PERCENT_RISE")

#Przycięcie mapy do granic gminy
nachylenie_przyciete = ExtractByMask(nachylenie,gmina)

#Utworzenie mapy przydatności terenu
nachylenie_kryterium6 = FuzzyMembership(nachylenie_przyciete,FuzzyLinear(21,10))

7.Dostęp do światła słonecznego: stoki południowe (SW-SE)

In [34]:
#Mapa kierunków nachylenia stoków
naslonecznienie = Aspect("gmina_swieradow_raster.tif")
naslonecznienie_przyciete = ExtractByMask(naslonecznienie,gmina)

#Opracowanie mapy przydatności kierunków nachylenia stoków
naslonecznienie_przydatnosc1 = FuzzyMembership(naslonecznienie_przyciete,FuzzyLinear(90,112.5))
naslonecznienie_przydatnosc2 = FuzzyMembership(naslonecznienie_przyciete,FuzzyLinear(270,247.5))
naslonecznienie_przydatnosc = naslonecznienie_przydatnosc1 * naslonecznienie_przydatnosc2

#Przypisanie płaskiemu terenowi o wartości nasłonecznienia -1 wartości 1 przydatności terenu
naslonecznienie_kryterium7 = arcpy.ia.Con(naslonecznienie_przyciete, 1, naslonecznienie_przydatnosc, "VALUE = -1")

8.Poza strefą ochronną gazociągu: min. 25m

In [ ]:
euclidean_distance = EucDistance("gazociag")
mapa_odleglosci = ExtractByMask(euclidean_distance,gmina)
gazociag_ostre = FuzzyMembership(mapa_odleglosci,FuzzyLinear(25,25.1))
gazociag_kryterium8 = FuzzyMembership(mapa_odleglosci,FuzzyLinear(25,300))

9.Użytkowanie terenu

In [35]:
#Stworzenie nowego atrybutu bool, który przypisze wartości 0/1 odpowiednim typom gleb
arcpy.management.CalculateField("gleby_gm_swieradow", "Usefulness", "selekcja(!TYP!)", "PYTHON3", """def selekcja(typ):
    if typ in ['G','E','M','T','F','Fb','Fc','FG']:
        return 0
    else:
        return 1""", "SHORT", "NO_ENFORCE_DOMAINS")

arcpy.conversion.FeatureToRaster("gleby_gm_swieradow", "Usefulness", r"mapa_gleb", 1)

<Result 'A:\\Studia\\analizy_przestrzenne\\MiloszWojciechowski\\projekt1\\Wyniki\\mapa_gleb'>

# Łączenie kryteriów

Łączenie kryteriów ostrych

In [9]:
#Połączenie warstw kryteriów ostrych
kryteria_ostre_lista = ["mapa_gleb", "lasy_kryterium5", "wody_ostre", "rzeki_ostre", "gazociag_ostre"]
kryteria_ostre = arcpy.sa.FuzzyOverlay(kryteria_ostre_lista, "AND", 0.9); kryteria_ostre.save(r"A:\Studia\analizy_przestrzenne\MiloszWojciechowski\projekt1\Wyniki\kryt_ostre")

RuntimeError: Nieprawidłowy wskaźnik. 

Łączenie kryteriów miękkich

In [10]:
#Różne wagi kryteriów
kryterium1 = ["hotele_kryterium1", "VALUE", 0.05]
kryterium2 = ["budynkiMieszkalne_kryterium2", "VALUE", 0.05]
kryterium3 = ["drogi_kryterium3", "VALUE", 0.2]
kryterium4Rzeki = ["rzeki_kryterium4", "VALUE", 0.1]
kryterium4Wody = ["wody_kryterium4", "VALUE", 0.15]
kryterium6 = ["nachylenie_kryterium6", "VALUE", 0.2]
kryterium7 = ["naslonecznienie_kryterium7", "VALUE", 0.1]
kryterium8 = ["gazociag_kryterium8", "VALUE", 0.15]

kryteria_miekkie_rozne = arcpy.ia.WeightedSum(WSTable([kryterium1, kryterium2,
                                                      kryterium3, kryterium4Rzeki,
                                                      kryterium4Wody, kryterium6,
                                                      kryterium7, kryterium8])); kryteria_miekkie_rozne.save(r"A:\Studia\analizy_przestrzenne\MiloszWojciechowski\projekt1\MyProject\MyProject.gdb\kryteria_miekkie_rozne")


#Te same wagi we wszystkich kryteriach
kryt1 = ["hotele_kryterium1", "VALUE", 0.125]
kryt2 = ["budynkiMieszkalne_kryterium2", "VALUE", 0.125]
kryt3 = ["drogi_kryterium3", "VALUE", 0.125]
kryt4Rzeki = ["rzeki_kryterium4", "VALUE", 0.125]
kryt4Wody = ["wody_kryterium4", "VALUE", 0.125]
kryt6 = ["nachylenie_kryterium6", "VALUE", 0.125]
kryt7 = ["naslonecznienie_kryterium7", "VALUE", 0.125]
kryt8 = ["gazociag_kryterium8", "VALUE", 0.125]

kryteria_miekkie_rowne = arcpy.ia.WeightedSum(WSTable([kryt1, kryt2, kryt3, kryt4Rzeki,
                                                      kryt4Wody, kryt6, kryt7, kryt8])); kryteria_miekkie_rozne.save(r"A:\Studia\analizy_przestrzenne\MiloszWojciechowski\projekt1\MyProject\MyProject.gdb\kryteria_miekkie_rowne")

Połączenie wszystkich kryteriów

In [11]:
#Połączenie kryteriów ostrych i miękkich kryteriów miękkich o różnych wagach
rozne_wagi = kryteria_ostre * kryteria_miekkie_rozne

#Połączenie kryteriów ostrych i miękkich kryteriów miękkich o takich samych wagach
rowne_wagi = kryteria_ostre * kryteria_miekkie_rowne

Reklasyfikacja map przydatności terenu od progu 0,8

In [12]:
#Reklasyfikacja dla różnych wag
reclassed_rozne = arcpy.sa.Reclassify("rozne_wagi", "VALUE", "0 0,8 0;0,8 1 1")

#Reklasyfikacja dla takich samych wag
reclassed_rowne = arcpy.sa.Reclassify("rowne_wagi", "VALUE", "0 0,8 0;0,8 1 1")

# Opracowanie kryteriów 10 i 11 oraz wynik

Stworzenie mapy kosztów względnych

In [13]:
#Dodanie wszystkich warstw pokrycia terenu do jednej listy
tab = ['PL.PZGiK.337.0210__OT_PTZB_A', 'PL.PZGiK.337.0210__OT_PTWZ_A', 'PL.PZGiK.337.0210__OT_PTWP_A', 'PL.PZGiK.337.0210__OT_PTUT_A', 'PL.PZGiK.337.0210__OT_PTTR_A', 'PL.PZGiK.337.0210__OT_PTSO_A', 'PL.PZGiK.337.0210__OT_PTRK_A', 'PL.PZGiK.337.0210__OT_PTPL_A', 'PL.PZGiK.337.0210__OT_PTNZ_A', 'PL.PZGiK.337.0210__OT_PTLZ_A', 'PL.PZGiK.337.0210__OT_PTKM_A', 'PL.PZGiK.337.0210__OT_PTGN_A']

#Połączenie wszystkich warstw pokrycia terenu w jedną
pokrycie_terenu = arcpy.management.Merge(tab, 'pokrycie_terenu')

#Dodanie do tabeli atrybutów nowego pola "koszt" z wartościami kosztów odpowiednimi dla każdego pokrycia
arcpy.management.CalculateField("pokrycie_terenu", "koszt", "temp(!X_KOD!)", "PYTHON3", """def temp(kod):
    dict = {'PTWP01':0,
    'PTWP02':200,
    'PTWP03':0,
    'PTZB01':200,
    'PTZB02':100,
    'PTZB03':200,
    'PTZB04':200,
    'PTZB05':50,
    'PTLZ01':100,
    'PTLZ02':50,
    'PTLZ03':50,
    'PTRK':15,
    'PTRK02':15,
    'PTUT01':0,
    'PTUT02':90,
    'PTUT03':100,
    'PTUT04':20,
    'PTUT05':20,
    'PTTR01':20,
    'PTTR02':1,
    'PTKM01':100,
    'PTKM02':200,
    'PTKM03':0,
    'PTKM04':0,
    'PTGN01':1,
    'PTGN02':1,
    'PTGN03':1,
    'PTGN04':1,
    'PTPL01':50,
    'PTSO01':0,
    'PTSO02':0,
    'PTWZ01':0,
    'PTWZ02':0,
    'PTNZ01':150,
    'PTNZ02':150}

    
    return dict[kod]

""", "SHORT", "NO_ENFORCE_DOMAINS")

#Zamiana warstwy wektorowej na raster
mapa_kosztow = arcpy.conversion.PolygonToRaster("pokrycie_terenu", "koszt", r"A:\Studia\analizy_przestrzenne\MiloszWojciechowski\projekt1\MyProject\MyProject.gdb\mapa_kosztow")

#Reklasyfikacja wartości mapy, żeby wartość 0 stała się wartością No Data
Mapa_kosztow_ost = arcpy.sa.Reclassify("mapa_kosztow", "Value", "0 NODATA;1 1;15 15;20 20;50 50;100 100;150 150;200 200", "DATA"); Mapa_kosztow_ost.save(r"A:\Studia\analizy_przestrzenne\MiloszWojciechowski\projekt1\MyProject\MyProject.gdb\Mapa_kosztow_ost")
Mapa_kosztow_ost = ExtractByMask(Mapa_kosztow_ost,gmina)

Kryterium 10: przydatne działki i obszary i 11:kształt obszaru - jak najbardziej zwarty

In [25]:
#Utworzenie kopii warstwy dzialek, na której będzie działać skrypt dla równych wartości wag
arcpy.management.CopyFeatures("dzialki", r"dzialki2")

<Result 'A:\\Studia\\analizy_przestrzenne\\MiloszWojciechowski\\projekt1\\Wyniki\\dzialki2.shp'>

Scenariusz dla różnych wartości wag

In [25]:
#Zamiana rastru z połączonymi kryteriami na poligony
poligon_rozne = arcpy.conversion.RasterToPolygon("reclassed_rozne", r"A:\Studia\analizy_przestrzenne\MiloszWojciechowski\projekt1\MyProject\MyProject.gdb\polygon_Reclass_rozne", "NO_SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)

#Wybór tylko tych poligonów, których przydatność wynosi 1
arcpy.management.SelectLayerByAttribute("polygon_Reclass_rozne", "NEW_SELECTION", "gridcode = 1")

#Stworzenie tabeli z atrybutem podającym procent przydantych ziem w stosunku do powierzchni działki
arcpy.analysis.TabulateIntersection("dzialki", "ID_DZIALKI", "polygon_Reclass_rozne", r"A:\Studia\analizy_przestrzenne\MiloszWojciechowski\projekt1\MyProject\MyProject.gdb\TabulateIntersection_rozne", None, None, None, "UNKNOWN")

#Dodanie do tabeli atrybutów warstwy działek tabeli z atrybutami z poprzedniego kroku
arcpy.management.AddJoin("dzialki", "ID_DZIALKI", "TabulateIntersection_rozne", "ID_DZIALKI", "KEEP_ALL", "NO_INDEX_JOIN_FIELDS")

#Wybór tych działek, w których pożądany teren stanowi min. 70% ich powierzchni
arcpy.management.SelectLayerByAttribute("dzialki", "NEW_SELECTION", "PERCENTAGE >= 70")

#Zapis pasujących działek do pliku
arcpy.management.CopyFeatures("dzialki", r"dobre_dzialki_rozne")

#Połączenie ze sobą sąsiadujących działek
arcpy.management.Dissolve("dobre_dzialki_rozne", r"A:\Studia\analizy_przestrzenne\MiloszWojciechowski\projekt1\MyProject\MyProject.gdb\dobre_dzialki_rozne_Dissolve",None, None, "SINGLE_PART")

#Wybranie tych działek, których powierzchnia jest większa od 1ha
arcpy.management.SelectLayerByAttribute("dobre_dzialki_rozne_Dissolve", "NEW_SELECTION", "Shape_Area >= 10000")

#Zapis pasujących działek do pliku
arcpy.management.CopyFeatures("dobre_dzialki_rozne_Dissolve", r"dzialki_rozne_min_1ha")

#Stworzenie mapy kosztów skumulowanych i mapy kierunków
koszty_skumulowane_rozne = arcpy.sa.CostDistance("dobre_dzialki_rozne_Dissolve", "Mapa_kosztow_ost")
koszty_skumulowane_rozne.save(r"A:\Studia\analizy_przestrzenne\MiloszWojciechowski\projekt1\MyProject\MyProject.gdb\koszty_skumulowane_rozne")

#Stworzenie mapy kierunków
mapa_kierunkiRozne = arcpy.sa.CostBackLink("dobre_dzialki_rozne_Dissolve", "Mapa_kosztow_ost")
mapa_kierunkiRozne.save(r"A:\Studia\analizy_przestrzenne\MiloszWojciechowski\projekt1\MyProject\MyProject.gdb\mapa_kierunkiRozne")

#Wyczyszczenie selecta
arcpy.SelectLayerByAttribute_management("polygon_Reclass_rozne", "CLEAR_SELECTION")
arcpy.SelectLayerByAttribute_management("dzialki", "CLEAR_SELECTION")

<Result 'dzialki'>

In [27]:
#Stworzenie najtańszego przyłącza do gazociągu
przylacze_gazociag_rozne = arcpy.sa.CostPath("gazociag", "koszty_skumulowane_rozne", "mapa_kierunkiRozne", "BEST_SINGLE", "Id", "INPUT_RANGE"); przylacze_gazociag_rozne.save(r"A:\Studia\analizy_przestrzenne\MiloszWojciechowski\projekt1\MyProject\MyProject.gdb\przylacze_gazociag_rozne")

In [26]:
#Dodanie atrybutu zawierającego stopień zwartości działki
arcpy.management.CalculateField("dzialki_rozne_min_1ha", "Compactnes", "(!Shape_Area!/(math.pi*(!Shape_Leng!/(2*math.pi))*2))*0.5",
                                "PYTHON3", None, "DOUBLE")

#Wybranie działki z najwyższą wartością współczynnika zwartości
arcpy.analysis.Select("dzialki_rozne_min_1ha", "dzialka_wynik_rozne_wagi", 
                      "Compactnes = (SELECT (MAX(Compactnes)) FROM dzialki_rozne_min_1ha)")

<Result 'dzialki_rozne_min_1ha'>

Scenariusz dla równych wartości wag

In [35]:
#Zamiana rastru z połączonymi kryteriami na poligony
poligon_rowne = arcpy.conversion.RasterToPolygon("reclassed_rowne", r"A:\Studia\analizy_przestrzenne\MiloszWojciechowski\projekt1\MyProject\MyProject.gdb\polygon_Reclass_rowne", "NO_SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)

#Wybór tylko tych poligonów, których przydatność wynosi 1
arcpy.management.SelectLayerByAttribute("polygon_Reclass_rowne", "NEW_SELECTION", "gridcode = 1")

#Stworzenie tabeli z atrybutem podającym procent przydantych ziem w stosunku do powierzchni działki
arcpy.analysis.TabulateIntersection("dzialki2", "ID_DZIALKI", "polygon_Reclass_rowne", r"A:\Studia\analizy_przestrzenne\MiloszWojciechowski\projekt1\MyProject\MyProject.gdb\TabulateIntersection_rowne", None, None, None, "UNKNOWN")

#Dodanie do tabeli atrybutów warstwy działek atrybutów z poprzedniego kroku
arcpy.management.AddJoin("dzialki2", "ID_DZIALKI", "TabulateIntersection_rowne", "ID_DZIALKI", "KEEP_ALL", "NO_INDEX_JOIN_FIELDS")

#Wybór tych działek, w których pożądany teren stanowi min. 70% ich powierzchni
arcpy.management.SelectLayerByAttribute("dzialki2", "NEW_SELECTION", "PERCENTAGE >= 70")

#Zapis pasujących działek do pliku
arcpy.management.CopyFeatures("dzialki2", r"dobre_dzialki_rowne")

#Połączenie ze sobą sąsiadujących działek
arcpy.management.Dissolve("dobre_dzialki_rozne", r"A:\Studia\analizy_przestrzenne\MiloszWojciechowski\projekt1\MyProject\MyProject.gdb\dobre_dzialki_rowne_Dissolve",None, None, "SINGLE_PART")

#Wybranie tych działek, których powierzchnia jest większa od 1ha
arcpy.management.SelectLayerByAttribute("dobre_dzialki_rowne_Dissolve", "NEW_SELECTION", "Shape_Area >= 10000")

#Zapis pasujących działek do pliku
arcpy.management.CopyFeatures("dobre_dzialki_rowne_Dissolve", r"dzialki_rowne_min_1ha")

#Stworzenie mapy kosztów skumulowanych i mapy kierunków
koszty_skumulowane_rowne = arcpy.sa.CostDistance("dobre_dzialki_rowne_Dissolve", "Mapa_kosztow_ost")
koszty_skumulowane_rowne.save(r"A:\Studia\analizy_przestrzenne\MiloszWojciechowski\projekt1\MyProject\MyProject.gdb\koszty_skumulowane_rowne")

#Stworzenie mapy kierunków
mapa_kierunkiRowne = arcpy.sa.CostBackLink("dobre_dzialki_rowne_Dissolve", "Mapa_kosztow_ost")
mapa_kierunkiRowne.save(r"A:\Studia\analizy_przestrzenne\MiloszWojciechowski\projekt1\MyProject\MyProject.gdb\mapa_kierunkiRowne")

#Wyczyszczenie selecta
arcpy.SelectLayerByAttribute_management("polygon_Reclass_rowne", "CLEAR_SELECTION")
arcpy.SelectLayerByAttribute_management("dzialki2", "CLEAR_SELECTION")

<Result 'dzialki2'>

In [23]:
#Stworzenie najtańszego przyłącza do gazociągu
przylacze_gazociag_rowne = arcpy.sa.CostPath("gazociag", "koszty_skumulowane_rowne", "mapa_kierunkiRowne", "BEST_SINGLE", "Id", "INPUT_RANGE"); przylacze_gazociag_rowne.save(r"A:\Studia\analizy_przestrzenne\MiloszWojciechowski\projekt1\MyProject\MyProject.gdb\przylacze_gazociag_rowne")

In [23]:
#Dodanie atrybutu zawierającego stopień zwartości działki
arcpy.management.CalculateField("dzialki_rozne_min_1ha", "Compactnes", "(!Shape_Area!/(math.pi*(!Shape_Leng!/(2*math.pi))*2))*0.5",
                                "PYTHON3", None, "DOUBLE")

#Wybranie działki z najwyższą wartością współczynnika zwartości
arcpy.analysis.Select("dzialki_rowne_min_1ha", "dzialka_wynik_rowne_wagi", 
                      "Compactnes = (SELECT (MAX(Compactnes)) FROM dzialki_rowne_min_1ha)")

<Result 'dzialki_rozne_min_1ha'>